In [1]:
from cv2 import normalize
import numpy as np
import cv2 as cv
import math
import time
import re
import os

In [2]:
def filter_pose_mask(array, mask):
    return [element for (i, element) in enumerate(array) if mask[i][0] == 1]

In [3]:
bf = cv.BFMatcher(cv.NORM_L1)

def featurematch_relloc(ref_data, query_data, camera_mtx, draw=False):
    (ref_img, (ref_kp, ref_desc)) = ref_data
    (query_img, (query_kp, query_desc)) = query_data

    # Match descriptors.
    matches = bf.knnMatch(ref_desc, query_desc, k=2)
    
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    if len(good_matches) < 5:
        return None, None

    good_ref_points = np.zeros(shape=(len(good_matches), 2), dtype=np.float32)
    good_query_points = np.zeros(shape=(len(good_matches), 2), dtype=np.float32)
    for i in range(len(good_matches)):
        match = good_matches[i]
        good_ref_points[i, 0] = ref_kp[match.queryIdx].pt[0]
        good_ref_points[i, 1] = ref_kp[match.queryIdx].pt[1]
        good_query_points[i, 0] = query_kp[match.trainIdx].pt[0]
        good_query_points[i, 1] = query_kp[match.trainIdx].pt[1]

    E, E_mask = cv.findEssentialMat(good_query_points, good_ref_points, camera_mtx, method=cv.RANSAC, prob=0.99, threshold=0.1, mask=None)
    retval, delta_R, delta_t, pose_mask = cv.recoverPose(E, good_query_points, good_ref_points, cameraMatrix=camera_mtx, mask=E_mask)

    if draw:
        img = cv.drawMatches(ref_img,ref_kp,query_img,query_kp,filter_pose_mask(good_matches, pose_mask),None,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS) 
        #img = draw_matches(img1, img2, good_old, good_new)
        cv.imshow('matches', img)
        cv.waitKey()

    return delta_t, delta_R

In [4]:
def intersect_rays(ao, ad, bo, bd):
    dx = bo[0] - ao[0]
    dy = bo[1] - ao[1]
    det = bd[0] * ad[1] - bd[1] * ad[0]
    u = (dy * bd[0] - dx * bd[1]) / det
    v = (dy * ad[0] - dx * ad[1]) / det
    
    return ao + ad * u

In [5]:
def unit_vector(vector):
    return vector / np.linalg.norm(vector)

# v1 and v2 must be normalized
def angle_between(v1, v2):
    return np.arccos(np.clip(np.dot(v1, v2), -1.0, 1.0))

In [6]:
def rotate_vector(v, r):
    rot = np.array([[math.cos(r), -math.sin(r)], [math.sin(r), math.cos(r)]])
    v_rot = np.dot(rot, v)
    return v_rot

In [7]:
def get_direction(frame_with_points, ref_idx, query_idx, angles, mtx, correct_angle=True, draw=False):
    t1, r1 = featurematch_relloc(frame_with_points[ref_idx], frame_with_points[query_idx], mtx, draw)
    if t1 is None or r1 is None:
        return None, None
    v = (t1[[0, 2], 0])
    if correct_angle:
        ref_r = angles[ref_idx]
        v_rot = rotate_vector(v, ref_r)
        return v_rot, r1
    else:
        return v, r1


In [8]:
def get_average_intersect(rays, positions):
    locations = []
    for (i1, v1) in enumerate(rays):
        for (i2, v2) in enumerate(rays[i1+1:]):
            if v1 is None or v2 is None:
                continue
            
            angle = angle_between(v1, v2)
            if angle < 0:
                angle = 2 * np.pi + angle

            p1 = positions[i1]
            p2 = positions[i2]
            location = intersect_rays(p1, v1, p2, v2)
            if np.dot(v1, location - p1) > 0 and np.dot(v2, location - p2) > 0:
                locations.append(location)

    valid = [location for location in locations if not np.isnan(location).any()]
    estimated_location = np.median(valid, axis=0)
    return estimated_location

In [9]:
def locate(ref_directions, ref_positions, plot=False, true_position=None):
    previous_estimate = None
    current_estimate = None
    has_data = len(ref_positions) > 0
    thresholds = [np.pi / 2, np.pi / 4, np.pi / 8, np.pi / 16]
    loop_num = 0
    
    current_estimate = get_average_intersect(ref_directions, ref_positions)
    while has_data and loop_num < len(thresholds):
        good_positions = []
        good_directions = []
        for (i, position) in enumerate(ref_positions):
            to_estimate = unit_vector(current_estimate - position)
            ref_ray = ref_directions[i]
            if ref_ray is not None:
                angle = angle_between(to_estimate, ref_ray)
                threshold = thresholds[loop_num]
                if angle < threshold or angle > (2 * np.pi) - threshold:
                    good_positions.append(position)
                    good_directions.append(ref_ray)

        ref_positions = good_positions
        ref_directions = good_directions
        loop_num = loop_num + 1
        has_data = len(ref_positions) > 0

        if not has_data:
            return current_estimate

        previous_estimate = current_estimate
        current_estimate = get_average_intersect(ref_directions, ref_positions)
        if np.isnan(current_estimate).any():
            return previous_estimate

        if plot:
            print(current_estimate)
            plot_rays(ref_directions, ref_positions, [current_estimate, true_position])

    return current_estimate

In [10]:
def compass_heading_to_angle(heading):
    # North is -135.6 degrees from the world x-axis
    # 0 degrees from the camera's perspective is straight forward along the camera y-axis so another 90 degrees to convert to camera x-axis
    correction_degrees = 135.6 + 90
    correction = np.deg2rad(correction_degrees)
    angle = heading - correction
    if angle < 0:
        angle = 2 * np.pi + angle
    return angle

In [11]:
def protractor_reading_to_angle(reading):
    # Protractor +tive is clock wise on the grid
    # Protractor 0 degrees is -90 degrees from the world x-axis
    # 0 degrees from the camera's perspective is straight forward along the camera y-axis so another 90 degrees to convert to camera x-axis
    reading = -reading
    reading = reading - (90 + 90)
    return np.deg2rad(reading)

In [12]:
with np.load('calibration.npz') as X:
    mtx, dist, _, _ = [X[i] for i in ('mtx','dist','rvecs','tvecs')]

In [13]:
base = "./protractor_positioned/"
paths = os.listdir(base)

angles = []
tagged_angles = []
positions = []
for path in paths:
    result = re.search("x(\d*\.\d*)_y(\d*\.\d*)_ch(\d*\.\d*)_t(-?\d*\.?\d*).png", path)
    if result:
        positions.append(np.array([float(result.group(1)), float(result.group(2))]))
        angles.append(compass_heading_to_angle(float(result.group(3))))
        tagged_angles.append(protractor_reading_to_angle(float(result.group(4))))

frames = [cv.cvtColor(cv.imread(base + path), cv.COLOR_BGR2GRAY) for path in paths]

first_frame = frames[0]
h, w = first_frame.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

undistorted = [cv.undistort(frame, mtx, dist, None, newcameramtx) for frame in frames]

detector = cv.ORB_create(10000)
descriptor = cv.xfeatures2d.BEBLID_create(0.75)

frame_with_points = [(frame, descriptor.compute(frame, detector.detect(frame, None))) for frame in undistorted]

In [14]:
import matplotlib.pyplot as plt

def plot_rays(directions, positions, additional_positions=[]):
    for (i, pos) in enumerate(positions):
        dir = directions[i]
        if dir is not None:
            dir = dir * 100
            x = [pos[0], pos[0] + dir[0]]
            y = [pos[1], pos[1] + dir[1]]
            plt.plot(x, y, marker='o')
            plt.plot(pos[0], pos[1], marker="^", ms=10)

    for positions in additional_positions:
        plt.plot(positions[0], positions[1], marker='x', ms=15)
    plt.xlim(0, 200)
    plt.ylim(-20, 200)
    plt.show()

In [15]:
paths

['x90.0_y60.0_ch1.127_t-22.5.png',
 'x90.0_y60.0_ch0.810_t-11.25.png',
 'x120.0_y60.0_ch6.259_t22.5.png',
 'x90.0_y60.0_ch0.276_t11.25.png',
 'x120.0_y60.0_ch0.996_t-11.25.png',
 'x90.0_y60.0_ch0.579_t0.png',
 'x90.0_y120.0_ch0.184_t22.5.png',
 'x120.0_y60.0_ch0.736_t-11.25.png',
 'x90.0_y60.0_ch1.109_t-22.5.png',
 'x90.0_y120.0_ch1.073_t-22.5.png',
 'x120.0_y60.0_ch0.504_t0.png',
 'x120.0_y90.0_ch0.790_t-11.25.png',
 'x120.0_y60.0_ch0.420_t22.5.png',
 'x90.0_y120.0_ch0.849_t-11.25.png',
 'x90.0_y60.0_ch0.739_t0.png',
 'x120.0_y60.0_ch1.144_t-22.5.png',
 'x120.0_y120.0_ch0.903_t-11.25.png',
 'x120.0_y90.0_ch1.042_t-22.5.png',
 'x120.0_y120.0_ch1.121_t-22.5.png',
 'x120.0_y90.0_ch0.259_t11.25.png',
 'x120.0_y120.0_ch0.693_t0.png',
 'x90.0_y60.0_ch0.921_t-11.25.png',
 'x90.0_y60.0_ch6.278_t22.5.png',
 'x120.0_y90.0_ch0.496_t0.png',
 'x90.0_y120.0_ch0.426_t11.25.png',
 'x90.0_y60.0_ch0.522_t11.25.png',
 'x90.0_y60.0_ch0.263_t22.5.png',
 'x120.0_y60.0_ch0.819_t0.png',
 'x120.0_y60.0_ch0.58

In [28]:
max_a = protractor_reading_to_angle(0)
min_a = protractor_reading_to_angle(12)

def location_query(frame_with_points, query, positions, angles, mtx, true_position=None):
    refs = [i for i in range(0, len(positions)) if i is not query]  
    ref_positions = []
    ref_directions = []
    ref_rotations = []
    for ref in refs:
        if angles[ref] > min_a and angles[ref] < max_a:
            ref_positions.append(positions[ref])
            dir, r = get_direction(frame_with_points, ref, query, angles, mtx, True, False)
            ref_directions.append(dir)
            ref_rotations.append(r)

    return locate(ref_directions, ref_positions, False, true_position)

In [29]:
query = 3
print(positions[query])
location = location_query(frame_with_points, query, positions, tagged_angles, mtx, positions[query])
location

[90. 60.]


array([87.69545042, 73.92569312])

In [88]:
compass_distances = []

for i in range(0, len(positions)):
    try:
        query = i
        estimate_position = location_query(frame_with_points, query, positions, angles, mtx)
        true_position = positions[query]
        distance = np.linalg.norm(true_position - estimate_position)
        compass_distances.append(distance)
        print(f"Guessed: {estimate_position} for {true_position}. Distance is {distance}")
    except:
        print(f"Failed")

Guessed: [63.67752555 56.27253689] for [90. 60.]. Distance is 26.585083082217178
Guessed: [65.77627363 62.59174645] for [90. 60.]. Distance is 24.361979990503453
Guessed: [109.10870247  37.15514025] for [120.  60.]. Distance is 25.30825910090893
Guessed: [76.2772309  57.07408189] for [90. 60.]. Distance is 14.031229044905361
Failed
Guessed: [80.15032445 69.09532416] for [90. 60.]. Distance is 13.406753150317966
Guessed: [147.77924784 233.44358485] for [ 90. 120.]. Distance is 127.3102054985322
Failed
Guessed: [70.35399181 64.30724787] for [90. 60.]. Distance is 20.11263339284103
Guessed: [101.88447708 134.48684941] for [ 90. 120.]. Distance is 18.73791880823857
Guessed: [112.57710471  47.58806192] for [120.  60.]. Distance is 14.462212193040534
Guessed: [123.51132334  82.35478414] for [120.  90.]. Distance is 8.413008807317105
Guessed: [110.09374023  52.09118472] for [120.  60.]. Distance is 12.676093316851052
Guessed: [ 97.48983057 128.26233006] for [ 90. 120.]. Distance is 11.1518455

/Users/deaston/.local/share/virtualenvs/RoboPose-93SmMfVE/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/deaston/.local/share/virtualenvs/RoboPose-93SmMfVE/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Guessed: [135.19352765 175.53150103] for [120. 120.]. Distance is 57.57248377983991
Guessed: [126.69301567  83.34390351] for [120.  90.]. Distance is 9.439283829103367
Guessed: [128.01403794 117.15876894] for [120. 120.]. Distance is 8.502787661741923
Guessed: [75.30133457 59.49614931] for [90. 60.]. Distance is 14.70729855934166
Guessed: [41.65789785 56.97965117] for [90. 60.]. Distance is 48.436363890019294
Guessed: [118.34793191  89.14905703] for [120.  90.]. Distance is 1.8583414385805939
Failed
Guessed: [72.95811683 56.54566464] for [90. 60.]. Distance is 17.388450610194063
Guessed: [77.04308007 57.58915956] for [90. 60.]. Distance is 13.179299133171641
Guessed: [79.32640814 36.12363618] for [120.  60.]. Distance is 47.163776610376694
Guessed: [116.54975008  49.5494597 ] for [120.  60.]. Distance is 11.005363108461072
Guessed: [ 97.59308716 130.26928724] for [ 90. 120.]. Distance is 12.771579113590326
Failed
Failed


In [93]:
np.percentile(distances, [10, 50, 70, 80, 90, 100])

array([  1.32981308,   4.8401543 ,   8.2610557 ,  15.99274894,
        87.18202225, 352.48086669])

In [86]:
np.mean(distances)

31.18529478527377

In [94]:
np.percentile(compass_distances, [10, 50, 70, 80, 90, 100])

array([  8.44892035,  14.03122904,  19.83769048,  25.5636239 ,
        47.92732898, 127.3102055 ])

In [95]:
np.mean(compass_distances)

23.12572881307819

: 